# Customer

In [1]:
import pandas as pd

## Database connection

In [3]:
import yaml
from sqlalchemy import create_engine


with open("../config.yml", "r") as file:
	config = yaml.safe_load(file)
	config_OLTP = config["OLTP"]
	config_OLAP = config["OLAP"]


url_OLTP = (f"{config_OLTP['drivername']}://{config_OLTP['user']}:{config_OLTP['password']}"
			f"@{config_OLTP['host']}:{config_OLTP['port']}/{config_OLTP['database_name']}")

url_OLAP = (f"{config_OLAP['drivername']}://{config_OLAP['user']}:{config_OLAP['password']}"
			f"@{config_OLAP['host']}:{config_OLAP['port']}/{config_OLAP['database_name']}")

OLTP_connection = create_engine(url_OLTP)
OLAP_connection = create_engine(url_OLAP)

## Extraction

In [41]:
client_dimension = pd.read_sql_table("cliente", OLTP_connection)
city_dimension = pd.read_sql_table("ciudad", OLTP_connection)

In [40]:
client_dimension.head(1)

,cliente_id,nit_cliente,nombre,email,direccion,telefono,nombre_contacto,ciudad_id,tipo_cliente_id,activo,coordinador_id,sector
0,1,25,Cliente 2,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,1,1,True,NaN,S


In [43]:
city_dimension.head(1)

,ciudad_id,nombre,departamento_id
0,6,BUGA,1


## Transformation

In [44]:
client_dimension = client_dimension.merge(city_dimension, on="ciudad_id", how="left")

In [45]:
client_dimension = client_dimension.rename(
    columns={
        "cliente_id": "customer_id",
        "nombre_x": "customer_name",
        "nombre_y": "customer_city",
    }
)

In [47]:
client_dimension.head(2)

,customer_id,nit_cliente,customer_name,email,direccion,telefono,nombre_contacto,ciudad_id,tipo_cliente_id,activo,coordinador_id,sector,customer_city,departamento_id
0,1,25,Cliente 2,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,1,1,True,NaN,S,CALI,1
1,2,123,Cliente 1,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,1,1,True,2.0,industrial,CALI,1


In [50]:
client_dimension = client_dimension[["customer_id", "customer_city"]]

In [51]:
client_dimension.head(2)

,customer_id,customer_city
0,1,CALI
1,2,CALI


## Load

In [ ]:
# Carga de datos
client_dimension.to_sql(
    "CUSTOMER_DIMENSION", OLAP_connection, if_exists="replace", index=False
)